In [1]:
import pandas as pd
import json
import os
import ast

In [2]:
LOCATIE_REQUESTS = os.path.join("../data/requests")
RUWE_DATA_CSV = os.path.join('../data/ruw/all_ruwe_data.csv')
GEKREGEN_EXCEL_FILE = os.path.join('../data/ModifiedQueryRows.xlsx')

In [3]:
def parse_filename(file_name) -> tuple:
    try:
        # rsplit verwijdert alleen de laatste .json extensie (voor mocht het bestand meerdere .json bevatten alhoewel dit onwaarschijnlijk lijkt)
        parts = file_name.rsplit('.json', 1)[0].split('-')
        if len(parts) >= 5:
            route_id = parts[0]
            date = parts[1]
            time = parts[2]
            number_of_tasks = parts[3]
            number_of_tasks_in_input_plan = parts[4]
            return route_id, date, time, number_of_tasks, number_of_tasks_in_input_plan
        else:
            print(f"Skipping file {file_name}: incorrect format")
            return None
    except Exception as e:
        print(f"Error parsing file name {file_name}: {e}")
        return None

In [4]:
# check if csv bestand bestaat
def check_csv_bestand(bestandspadennaam) -> bool:
    if os.path.exists(bestandspadennaam):
        return True
    return False

# lees csv bestand in als dataframe
def lees_dataframe_uit_csv(bestandspadennaam) -> pd.DataFrame:
    df = pd.read_csv(bestandspadennaam)  
    return df

# maak csv bestand aan vanuit dataframe
def schrijf_dataframe_naar_csv(df: pd.DataFrame, bestandspadennaam) -> None:
    # Maak de directory aan als deze niet bestaat
    os.makedirs(os.path.dirname(bestandspadennaam), exist_ok=True)
    # Schrijf het dataframe naar CSV
    df.to_csv(bestandspadennaam, index=False)
    return None

# de verschillende json-bestanden van de requests inladen en samenvoegen tot 1 grote dataframe 
# geeft een dataframe terug als resultaat
def lees_json_bestanden_en_maak_dataframe(locatie_requests) -> pd.DataFrame:
    df = pd.DataFrame()
    for folder_name in os.listdir(locatie_requests):
        folder_path = os.path.join(locatie_requests, folder_name)
        if os.path.isdir(folder_path):
            for file_name in os.listdir(folder_path):
                if file_name.endswith('.json'):
                    parsed_data = parse_filename(file_name)
                    if parsed_data == None:
                        break
                    else:
                        route_id, date, time, number_of_tasks, number_of_tasks_in_input_plan = parsed_data
                        # print(route_id, date, time, number_of_tasks, number_of_tasks_in_input_plan)
                        file_path = os.path.join(folder_path, file_name)
                        with open(file_path, 'r') as f:
                            data = json.load(f)
                            # Voeg route_id toe aan elke record in data
                            data['route_id'] = route_id
                            data['date'] = date
                            data['time'] = time
                            data['number_of_tasks'] = number_of_tasks
                            data['number_of_tasks_in_input_plan'] = number_of_tasks_in_input_plan
                            temp_df = pd.DataFrame([data])
                            # voeg tijdelijke dataframe toe aan de hoofddataframe
                            df = pd.concat([df, temp_df], ignore_index=True)
    
    return df


In [17]:

if (check_csv_bestand(RUWE_DATA_CSV) == False):
    # maak het bestand een eerste keer

    ingelezen_dataframe = lees_json_bestanden_en_maak_dataframe(LOCATIE_REQUESTS)
    # dit is dan dezelfde dataframe als in de gegeven excel file maar met tasks en fixedTasks eraan toegevoegd en zonder TriggerType
    # date en time zijn nog steeds strings, kan later nog omgezet worden naar datetime indien nodig
    # timecalculation, 
    display(ingelezen_dataframe.head())

    schrijf_dataframe_naar_csv(ingelezen_dataframe, RUWE_DATA_CSV)

    display(ingelezen_dataframe.tail())
    print(f"Lengte van de dataframe: {len(ingelezen_dataframe)}")
    # excel inlezen om de lengte te checken
    df_excel = pd.read_excel(GEKREGEN_EXCEL_FILE)
    print(f"Aantal rijen in ingelezen excel dataframe: {len(df_excel)}")
    if (len(ingelezen_dataframe) != len(df_excel)):
        print("Waarschuwing: Aantal rijen in ingelezen dataframe komt niet overeen met aantal rijen in excel dataframe!")

else:
    # lees het bestand en voeg toe aan dataframe om verder mee te werken
    ingelezen_dataframe = lees_dataframe_uit_csv(RUWE_DATA_CSV)
    display(ingelezen_dataframe.tail())


,id,configurationName,tasks,fixedTasks,route_id,date,time,number_of_tasks,number_of_tasks_in_input_plan
21701,642acef2-3942-4057-aeb8-61df9f7ffa1b,CreateSequence,"[{'id': '63784', 'address': {'latitude': 0.482...",[],0521_O69,20220622,64101,135,0
21702,2d4fe424-cf93-49ea-afdc-6475b244fe69,CreateSequence,"[{'id': '63759', 'address': {'latitude': 0.487...",[],0521_O69,20220622,64811,139,0
21703,0e9bc064-e6ee-421b-ae3f-e09b39f6cf86,CreateSequence,"[{'id': '63652', 'address': {'latitude': 0.485...",[],0521_O69,20220622,72840,139,0
21704,c3b3eea2-dff2-4680-9002-b49ee714ca2d,EstimateTime,"[{'id': '63788', 'address': {'latitude': 0.487...","[{'taskId': '63788', 'activityType': 'Task', '...",0521_O69,20220622,73145,139,139
21705,8a8e6d6b-045c-44bd-9085-cf2d5ec7a6d1,EstimateTime,"[{'id': '63788', 'address': {'latitude': 0.487...","[{'taskId': '63788', 'activityType': 'Task', '...",0521_O69,20220622,83833,139,139


In [42]:
# dataframe met tasks maken
taken_df = ingelezen_dataframe[['id', 'tasks']].copy()
# type van tasks is een string dus moeten we dit eerst omzetten
def safe_eval(x):
    if pd.isna(x) or x == '':
        return []
    try:
        return ast.literal_eval(x)
    except Exception:
        return []


taken_df['tasks'] = taken_df['tasks'].apply(safe_eval)
taken_df = taken_df.explode('tasks')
tasks_normalized = pd.json_normalize(taken_df['tasks'])

tasks_normalized = tasks_normalized.rename(columns={
    'id': 'task_id',
    'address.latitude': 'latitude',
    'address.longitude': 'longitude',
    'timeWindow.from': 'from',
    'timeWindow.till': 'till'
})
tasks_normalized['id'] = taken_df['id'].values
tasks_normalized = tasks_normalized.reset_index(drop=True)

display(tasks_normalized.head())


,task_id,latitude,longitude,from,till,id
0,395,0.565826,0.221868,2022-06-17T08:00:00,2022-06-17T17:00:00,41931cd2-8975-4a64-9197-d16abe871bb7
1,394,0.565826,0.221868,2022-06-17T07:30:00,2022-06-17T18:00:00,41931cd2-8975-4a64-9197-d16abe871bb7
2,385,0.565826,0.221868,2022-06-17T08:00:00,2022-06-17T17:00:00,5737d4ef-23e9-4832-8bb5-24340a176e31
3,384,0.565826,0.221868,2022-06-17T07:30:00,2022-06-17T23:59:00,5737d4ef-23e9-4832-8bb5-24340a176e31
4,390,0.565826,0.221868,2022-06-17T08:00:00,2022-06-17T17:00:00,34f409b3-9f92-4720-9926-48ca1cbf6f90


In [ ]:
# dataframe met fixedTasks maken
fixed_taken_df = ingelezen_dataframe[['id', 'fixedTasks']].copy()
# filter om alleen rijen te tonen waar fixedTasks niet leeg is
fixed_taken_df = fixed_taken_df[fixed_taken_df['fixedTasks'].astype(str) != '[]']

fixed_taken_df['fixedTasks'] = fixed_taken_df['fixedTasks'].apply(safe_eval)
# fixedTasks omzetten van string naar object
fixed_taken_df = fixed_taken_df.explode('fixedTasks')

# alleen rijen behouden waar fixedTasks niet leeg is
fixed_taken_df = fixed_taken_df[fixed_taken_df['fixedTasks'].notna()]
fixed_taken_df = fixed_taken_df[fixed_taken_df['fixedTasks'] != {}]

# normaliseren van fixedTasks
fixed_tasks_normalized = pd.json_normalize(fixed_taken_df['fixedTasks'])

# kolommen hernoemen voor consistentie
if not fixed_tasks_normalized.empty:
    # voeg id kolom toe
    fixed_tasks_normalized['id'] = fixed_taken_df['id'].values
    fixed_tasks_normalized = fixed_tasks_normalized.reset_index(drop=True)

display(fixed_tasks_normalized.head())